In [1]:
import plotly.graph_objects as go 
from plotly.subplots import make_subplots
import aerosandbox.numpy as anp
import aerosandbox as asb 
from surrogate import Surrogate2Callback,Surrogate,KRG

In [2]:
xt = anp.array([0.0, 1.0, 2.0, 3.0, 4.0])
yt = anp.array([0.0, 1.0, 1.5, 0.9, 1.0])

sm = KRG(theta0=[1e-2],print_global=False)
sm.set_training_values(xt, yt)
sm.train()

sm = Surrogate(sm)
sm_predict = Surrogate2Callback("sm", surrogate=sm)

num = 100
x = anp.linspace(0.0, 4.0, num)
y=sm_predict(x).toarray()

fig=make_subplots(1,1)
fig.add_scatter(x=x,y=y[:,0],mode="lines+markers",row=1,col=1,line={"shape":"spline","width":2})
fig.update_layout(width=600,margin={"r":20,"l":20,"t":20,"b":20})
fig.show(config={"scrollZoom":True})

In [ ]:
opti=asb.Opti()
x=opti.variable(init_guess=1.0,lower_bound=0.0,upper_bound=4.0)
obj=sm_predict(x)
opti.maximize(obj)
sol=opti.solve()
print(sol(x),sol(obj))

In [ ]:
from scipy.stats import norm 

def norm_pdf(x,loc=0,scale=1):
    x=(x-loc)/scale 
    y=anp.exp(-x**2/2)/(anp.sqrt(2*anp.pi))
    y=y/scale 
    return y 


x=5 
print(norm.pdf(x))
print(norm_pdf(x))
norm.cdf

1.4867195147342979e-06
1.4867195147342979e-06


In [4]:
import casadi as ca

# 定义变量
t = ca.MX.sym('t')   # 时间
x = ca.MX.sym('x')   # 状态（这里状态就是我们要累积的积分值）

# 定义被积函数，例如 f(t) = t^2
f = t**2

# 定义微分方程：dx/dt = f(t)
ode = {'x': x, 't': t, 'ode': f}

# 创建积分器，区间 [a, b]
a = 0
b = 2
I = ca.integrator('I', 'cvodes', ode, {'t0': a, 'tf': b})

# 初始状态（积分从0开始）
x0 = ca.MX(0)
res = I(x0=x0)

# 积分结果
result = res['xf']
print("积分值:", result)

积分值: @1=0x0, I(0, 0x1, 0x1, 0x1, @1, @1, @1){0}


CasADi - 2025-06-29 21:47:54 WARNING("The options 't0', 'tf', 'grid' and 'output_t0' have been deprecated.
The same functionality is provided by providing additional input arguments to the 'integrator' function, in particular:
 * Call integrator(..., t0, tf, options) for a single output time, or
 * Call integrator(..., t0, grid, options) for multiple grid points.
The legacy 'output_t0' option can be emulated by including or excluding 't0' in 'grid'.
Backwards compatibility is provided in this release only.") [.../casadi/core/integrator.cpp:692]
